In [176]:
import pandas as pd
import sqlalchemy 
import sql_functions as sf

In [168]:
import matplotlib.pyplot as plt

In [26]:
# load jobs data

schema = 'capstone_datacvpro'

jobs_20 = sf.get_dataframe(f' SELECT * FROM {schema}.jobs_20')
display(jobs_20)

,job_title,salary_estimate,job_description,company_name,location,industry
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,"New York, NY",Social Assistance
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,"New York, NY",Health Care Services & Hospitals
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,Squarespace\n3.4,"New York, NY",Internet
3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,"New York, NY",IT Services
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,"New York, NY",Sports & Recreation
...,...,...,...,...,...,...
6157,AWS Data Engineer,$55K-$112K (Glassdoor est.),About Us\n\nTachyon Technologies is a Digital ...,Tachyon Technologies\n4.4,"Dublin, OH",IT Services
6158,Data Analyst â Junior,$55K-$112K (Glassdoor est.),"Job description\nInterpret data, analyze resul...","Staffigo Technical Services, LLC\n5.0","Columbus, OH",IT Services
6159,Security Analytics Data Engineer,$55K-$112K (Glassdoor est.),Job DescriptionThe Security Analytics Data Eng...,"PDS Tech, Inc.\n3.8","Dublin, OH",Staffing & Outsourcing
6160,Security Analytics Data Engineer,$55K-$112K (Glassdoor est.),The Security Analytics Data Engineer will inte...,Data Resource Technologies\n4.0,"Dublin, OH",Accounting


In [180]:
import re
import nltk
import spacy

In [31]:
# Steps in NLP:

# Tokenizing: split text by word/ sentence 
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

# Stop Words: filter out word that are not needed for context (in, is, and,...)
nltk.download("stopwords")
from nltk.corpus import stopwords

# Stemming: reduce words to their root
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Tagging Parts of Speech: labeling words (noun, pronoun, verb,..)

# Lemmatizing: reduce word to their core meaning
from nltk.stem import WordNetLemmatizer

# Chunking: identify phrases ,word need to be tagged before chunking
from nltk.tokenize import word_tokenize

# Named Entity Recognition: identify named entities, eg. locations, people, organizations,..

[nltk_data] Downloading package punkt to /Users/kgolzev/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kgolzev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [77]:
# count words in description column

def count_words(text):
    words = word_tokenize(text)
    return len(words)

jobs_20['desc_words'] = jobs_20['job_description'].apply(count_words)

In [53]:
# count sentences

def count_sent(text):
    sent = sent_tokenize(text)
    return len(sent)

jobs_20['desc_sent'] = jobs_20['job_description'].apply(count_sent)

In [54]:
jobs_20.head()

,job_title,salary_estimate,job_description,company_name,location,industry,desc_words,desc_sent
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,"New York, NY",Social Assistance,1162,48
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,"New York, NY",Health Care Services & Hospitals,709,48
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,Squarespace\n3.4,"New York, NY",Internet,802,24
3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,"New York, NY",IT Services,888,28
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,"New York, NY",Sports & Recreation,573,13


In [55]:
jobs_20.describe()

,desc_words,desc_sent
count,6162.000000,6162.000000
mean,552.812074,18.711782
std,321.742581,13.699206
min,3.000000,1.000000
25%,320.000000,9.000000
50%,510.000000,16.000000
75%,726.000000,26.000000
max,3420.000000,106.000000


In [213]:
# remove stopwords & convert to lower case

def no_stopwords(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    word_list = []
    for word in words:
        if word.lower() not in stop_words and word.isalpha():   # isalpha checks if characters are alphabetic letters
            word_list.append(word.lower())
    return(word_list)

jobs_20['desc_cleaned'] = jobs_20['job_description'].apply(no_stopwords)

In [203]:
# do not use, gives back high count of words
# remove stopwords and not create list 

def no_stopwords(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    clean = ' '.join([word for word in words if word not in (stop_words)])
    return(clean)

jobs_20['desc_cleaned'] = jobs_20['job_description'].apply(no_stopwords)

In [214]:
jobs_20.head()

,job_title,salary_estimate,job_description,company_name,location,industry,desc_words,desc_sent,desc_cleaned,desc_words_clean,word_counts,extracted_skills,skills_spacy,skills_predefined
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,"New York, NY",Social Assistance,832,48,"[eager, roll, sleeves, harness, data, drive, p...",630,"[(data, 13), (Vera, 9), (projects, 8), (workin...",[],"[policy, data, management, demonstration, exam...",[]
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,"New York, NY",Health Care Services & Hospitals,550,48,"[overview, provides, analytical, technical, su...",422,"[(data, 18), (Quality, 9), (required, 9), (Par...",[],"[data, business, data, Monitors, data, quality...","[data analysis, data analysis, communication]"
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,Squarespace\n3.4,"New York, NY",Internet,531,24,"[looking, senior, data, analyst, love, mentors...",402,"[(team, 12), (data, 10), (insights, 6), (Squar...",[],"[data, data, Product, data, care, customer, da...",[]
3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,"New York, NY",IT Services,643,28,"[requisition, remote, yes, collaborate, create...",459,"[(Celerity, 11), (business, 9), (data, 8), (wo...",[],"[candidate, data, data, business, operations, ...","[teamwork, data analysis, communication]"
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,"New York, NY",Sports & Recreation,399,13,"[fanduel, group, fanduel, group, team, brands,...",312,"[(FanDuel, 9), (team, 6), (sports, 4), (regula...",[],"[world, class, leave, money, fantasy, sports, ...",[data analysis]


In [215]:
# count words in cleaned description

def count_words_list(list):
    return len(list)

jobs_20['desc_words_clean'] = jobs_20['desc_cleaned'].apply(count_words_list)

In [216]:
jobs_20.head()

,job_title,salary_estimate,job_description,company_name,location,industry,desc_words,desc_sent,desc_cleaned,desc_words_clean,word_counts,extracted_skills,skills_spacy,skills_predefined
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,"New York, NY",Social Assistance,832,48,"[eager, roll, sleeves, harness, data, drive, p...",630,"[(data, 13), (Vera, 9), (projects, 8), (workin...",[],"[policy, data, management, demonstration, exam...",[]
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,"New York, NY",Health Care Services & Hospitals,550,48,"[overview, provides, analytical, technical, su...",422,"[(data, 18), (Quality, 9), (required, 9), (Par...",[],"[data, business, data, Monitors, data, quality...","[data analysis, data analysis, communication]"
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,Squarespace\n3.4,"New York, NY",Internet,531,24,"[looking, senior, data, analyst, love, mentors...",402,"[(team, 12), (data, 10), (insights, 6), (Squar...",[],"[data, data, Product, data, care, customer, da...",[]
3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,"New York, NY",IT Services,643,28,"[requisition, remote, yes, collaborate, create...",459,"[(Celerity, 11), (business, 9), (data, 8), (wo...",[],"[candidate, data, data, business, operations, ...","[teamwork, data analysis, communication]"
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,"New York, NY",Sports & Recreation,399,13,"[fanduel, group, fanduel, group, team, brands,...",312,"[(FanDuel, 9), (team, 6), (sports, 4), (regula...",[],"[world, class, leave, money, fantasy, sports, ...",[data analysis]


In [209]:
jobs_20.describe()

,desc_words,desc_sent,desc_words_clean
count,6162.000000,6162.000000,6162.000000
mean,411.268257,18.711782,308.648815
std,235.412094,13.699206,173.846100
min,3.000000,1.000000,3.000000
25%,240.000000,9.000000,182.000000
50%,381.000000,16.000000,287.000000
75%,536.000000,26.000000,400.000000
max,2261.000000,106.000000,1711.000000


In [217]:
# count occurence of words & show top 5

from collections import Counter

def counter(words):
    return Counter(words).most_common(5)

jobs_20['word_counts'] = jobs_20['desc_cleaned'].apply(counter)

In [218]:
jobs_20.head()

,job_title,salary_estimate,job_description,company_name,location,industry,desc_words,desc_sent,desc_cleaned,desc_words_clean,word_counts,extracted_skills,skills_spacy,skills_predefined
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,"New York, NY",Social Assistance,832,48,"[eager, roll, sleeves, harness, data, drive, p...",630,"[(data, 19), (justice, 11), (vera, 9), (workin...",[],"[policy, data, management, demonstration, exam...",[]
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,"New York, NY",Health Care Services & Hospitals,550,48,"[overview, provides, analytical, technical, su...",422,"[(data, 18), (quality, 13), (required, 9), (ma...",[],"[data, business, data, Monitors, data, quality...","[data analysis, data analysis, communication]"
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,Squarespace\n3.4,"New York, NY",Internet,531,24,"[looking, senior, data, analyst, love, mentors...",402,"[(data, 14), (team, 12), (insights, 8), (custo...",[],"[data, data, Product, data, care, customer, da...",[]
3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,"New York, NY",IT Services,643,28,"[requisition, remote, yes, collaborate, create...",459,"[(business, 11), (celerity, 11), (data, 10), (...",[],"[candidate, data, data, business, operations, ...","[teamwork, data analysis, communication]"
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,"New York, NY",Sports & Recreation,399,13,"[fanduel, group, fanduel, group, team, brands,...",312,"[(fanduel, 10), (team, 6), (reporting, 5), (da...",[],"[world, class, leave, money, fantasy, sports, ...",[data analysis]


In [221]:

# combine the list in desc_cleaned of each row into one list
all_words = [word for sublist in jobs_20['desc_cleaned'] for word in sublist]

# Count the occurrences of each word
word_counts = Counter(all_words)

# Get the most common tokens
most_common_words = word_counts.most_common(30)

# Print the most common tokens
for word, count in most_common_words:
    print(f'{word}: {count}')

data: 58911
experience: 29776
business: 16561
work: 15186
skills: 11409
team: 11053
analysis: 9084
years: 9051
ability: 8654
development: 8030
management: 7628
knowledge: 7464
analytics: 7064
job: 7053
required: 6958
information: 6944
science: 6900
strong: 6793
support: 6644
working: 6526
including: 6426
solutions: 6336
requirements: 6308
new: 6161
technical: 5965
tools: 5695
research: 5602
systems: 5591
sql: 5251
design: 5240


In [170]:
### NOT WORKING ###

nlp = spacy.load("en_core_web_sm")

#function to extract skills using spacy
def extract_skills(text):
    doc = nlp(text)
    skills = [ent.text.lower() for ent in doc.ents if ent.label_ == "SKILL"]
    return list(set(skills))

# Apply the function to create a new column
jobs_20['extracted_skills'] = jobs_20['job_description'].apply(extract_skills)

In [188]:
# check for predefined list of skills

from spacy.matcher import PhraseMatcher

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Define a list of skills
skills_list = ["python", "java", "machine learning", "data analysis", "communication", "teamwork"]  # add desired skills

# Initialize phrase matcher
matcher = PhraseMatcher(nlp.vocab)
patterns = [nlp(skill) for skill in skills_list]
matcher.add("SKILLS", None, *patterns)

# Function to extract skills using spaCy and predefined list
def extract_skills_predefined(text):
    doc = nlp(text)
    matches = matcher(doc)
    skills = [doc[start:end].text for match_id, start, end in matches]
    return skills

# Assuming you have a DataFrame named 'jobs_20' with a column 'job_descriptions'
jobs_20['skills_predefined'] = jobs_20['job_description'].apply(extract_skills_predefined)

In [189]:
jobs_20.head()

,job_title,salary_estimate,job_description,company_name,location,industry,desc_words,desc_sent,desc_cleaned,desc_words_clean,word_counts,extracted_skills,skills_spacy,skills_predefined
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,"New York, NY",Social Assistance,832,48,Are eager roll sleeves harness data drive poli...,5899,"[(,, 62), (., 44), (data, 13), (Vera, 9), (:, 9)]",[],"[policy, data, management, demonstration, exam...",[]
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,"New York, NY",Health Care Services & Hospitals,550,48,Overview Provides analytical technical support...,4200,"[(,, 48), (., 47), (data, 18), (Quality, 9), (...",[],"[data, business, data, Monitors, data, quality...","[data analysis, data analysis, communication]"
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,Squarespace\n3.4,"New York, NY",Internet,531,24,We ’ looking Senior Data Analyst love mentorsh...,3767,"[(,, 52), (., 23), (team, 12), (’, 11), (data,...",[],"[data, data, Product, data, care, customer, da...",[]
3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,"New York, NY",IT Services,643,28,Requisition NumberRR-0001939 Remote : Yes We c...,4458,"[(,, 73), (., 25), (•, 15), (Celerity, 11), (b...",[],"[candidate, data, data, business, operations, ...","[teamwork, data analysis, communication]"
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,"New York, NY",Sports & Recreation,399,13,ABOUT FANDUEL GROUP FanDuel Group world-class ...,2771,"[(,, 19), (., 13), (FanDuel, 9), (—, 8), (team...",[],"[world, class, leave, money, fantasy, sports, ...",[data analysis]


In [181]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to extract skills using spaCy
def extract_skills_spacy(text):
    doc = nlp(text)
    skills = []
    for token in doc:
        if token.pos_ == "NOUN" and token.dep_ == "compound":
            skills.append(token.text)
    return skills

jobs_20['skills_spacy'] = jobs_20['job_description'].apply(extract_skills_spacy)

In [ ]:
from nltk.tag import pos_tag



In [183]:
jobs_20.head()

,job_title,salary_estimate,job_description,company_name,location,industry,desc_words,desc_sent,desc_cleaned,desc_words_clean,word_counts,extracted_skills,skills_spacy
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,"New York, NY",Social Assistance,832,48,Are eager roll sleeves harness data drive poli...,5899,"[(,, 62), (., 44), (data, 13), (Vera, 9), (:, 9)]",[],"[policy, data, management, demonstration, exam..."
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,"New York, NY",Health Care Services & Hospitals,550,48,Overview Provides analytical technical support...,4200,"[(,, 48), (., 47), (data, 18), (Quality, 9), (...",[],"[data, business, data, Monitors, data, quality..."
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,Squarespace\n3.4,"New York, NY",Internet,531,24,We ’ looking Senior Data Analyst love mentorsh...,3767,"[(,, 52), (., 23), (team, 12), (’, 11), (data,...",[],"[data, data, Product, data, care, customer, da..."
3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,"New York, NY",IT Services,643,28,Requisition NumberRR-0001939 Remote : Yes We c...,4458,"[(,, 73), (., 25), (•, 15), (Celerity, 11), (b...",[],"[candidate, data, data, business, operations, ..."
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,"New York, NY",Sports & Recreation,399,13,ABOUT FANDUEL GROUP FanDuel Group world-class ...,2771,"[(,, 19), (., 13), (FanDuel, 9), (—, 8), (team...",[],"[world, class, leave, money, fantasy, sports, ..."
